<a href="https://colab.research.google.com/github/arya2910/Next-word-Text-Generation/blob/main/text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Embedding

In [ ]:
text="I do well in school, and people think I am smart because of it. But it’s not true. In fact, three years ago I struggled in school. However, two years ago I decided to get serious about school and made a few changes. First, I decided I would become interested in whatever was being taught, regardless of what other people thought. I also decided I would work hard every day and never give up on any assignment. I decided to never, never fall behind. Finally, I decided to make school a priority over friends and fun. After implementing these changes, I became an active participant in classroom discussions. Then my test scores began to rise. I still remember the first time that someone made fun of me because “I was smart.” How exciting! It seems to me that being smart is simply a matter of working hard and being interested. After all, learning a new video game is hard work even when you are interested. Unfortunately, learning a new video game doesn’t help you get into college or get a good job."

In [ ]:
# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
sequences = tokenizer.texts_to_sequences([text])[0]

In [ ]:
# Generate input sequences and labels
X, y = [], []
for i in range(1, len(sequences)):
    X.append(sequences[:i])
    y.append(sequences[i])
y = to_categorical(y, num_classes=len(tokenizer.word_index)+1)


In [ ]:
# Pad the input sequences
maxlen = max([len(seq) for seq in X])
X = pad_sequences(X, maxlen=maxlen, padding='pre')

In [ ]:
# Build the model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=50, input_length=maxlen))
model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(len(tokenizer.word_index)+1, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(X, y, epochs=100, verbose=2)

In [ ]:
seed_text = 'i am smart'
for i in range(10):
    sequence = tokenizer.texts_to_sequences([seed_text])[0]
    padded_sequence = pad_sequences([sequence], maxlen=maxlen, padding='post')
    predicted_word_index = np.argmax(model.predict(padded_sequence), axis=-1)[0]
    predicted_word = tokenizer.index_word[predicted_word_index]
    seed_text += ' ' + predicted_word
print(seed_text)